In [1]:
import sys
import os

sys.path.append(os.path.abspath(".."))

from final_code.pubmed_api import pubmed_api_pull
from final_code.language_model import check_if_text_has_outbreak
from final_code.language_model import extract_outbreak_place
import pandas as pd
import requests

In [4]:
t = ["Avian Influenza outbreak"]
n = 3

In [5]:
df = pubmed_api_pull(t, n)
df

,pmid,title,abstract,full_text,authors
0,29522738,Making sense of the virome in light of evoluti...,Understanding the patterns and drivers of vira...,1 . Introduction Viruses are ubiquitous across...,https://orcid.org/0000-0001-5367-420X Wallace ...
1,40053686,PathoSeq-QC: a decision support bioinformatics...,AbstractMotivationRecommendations on the use o...,1 Introduction Technology development in the l...,https://orcid.org/0000-0002-4899-5284 Leoni Ga...
2,40036232,History of scrub typhus in Indonesia,AbstractScrub typhus is a common but underreco...,Introduction Scrub typhus is an underrecognize...,https://orcid.org/0000-0001-8705-9176 Saraswat...


In [6]:
df["has_outbreak"] = df["full_text"].apply(check_if_text_has_outbreak)
df

,pmid,title,abstract,full_text,authors,has_outbreak
0,29522738,Making sense of the virome in light of evoluti...,Understanding the patterns and drivers of vira...,1 . Introduction Viruses are ubiquitous across...,https://orcid.org/0000-0001-5367-420X Wallace ...,1. This text discusses various factors that in...
1,40053686,PathoSeq-QC: a decision support bioinformatics...,AbstractMotivationRecommendations on the use o...,1 Introduction Technology development in the l...,https://orcid.org/0000-0002-4899-5284 Leoni Ga...,1. Abstract:\nThe manuscript presents PathoSeq...
2,40036232,History of scrub typhus in Indonesia,AbstractScrub typhus is a common but underreco...,Introduction Scrub typhus is an underrecognize...,https://orcid.org/0000-0001-8705-9176 Saraswat...,This text provides an overview of the history...
